In [1]:
import sys
import os
%matplotlib inline
import matplotlib.pyplot as plt
plt.clf()
import glob
from transformers import AutoFeatureExtractor, ASTForAudioClassification, ASTFeatureExtractor
import torch
sys.path.append(r"C:\Users\NoahB\OneDrive\Desktop\cetacean_detection")
from cetacean_detection.src.data.ast_dataset import generate_ast_dataset
from cetacean_detection.src.data.pre_processor import PreProcessConfig
from cetacean_detection.utils.config import Config, GeneralConfig
config = r"C:\Users\NoahB\OneDrive\Desktop\cetacean_detection\cetacean_detection\configs\pipeline.yaml"
configs = Config.from_yaml(config)
config: PreProcessConfig = configs.get("preprocess", PreProcessConfig())


<Figure size 640x480 with 0 Axes>

In [2]:
import numpy as np
wav_dir = r"C:\Users\NoahB\OneDrive\Desktop\cetacean_detection\nefsc_sbnms_200903_nopp6_ch10\source-audio"
wav_files = glob.glob(os.path.join(wav_dir, "*.wav"))
sampling_rate, X, y = generate_ast_dataset(wav_files[0], config)
# the feature extractor can be geussed using the auto feature extractor
# feature_extractor = AutoFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593", sampling_rate=sampling_rate, num_mel_bins=40, max_length=40)

feature_extractor = ASTFeatureExtractor(
    sampling_rate=2000,
    num_mel_bins=128,  # Adjust as needed
    max_length=58,
    do_normalize=False
)



positive_entry = np.array(X)[np.array(y) == 1][1]
print(positive_entry.shape)
negative_entry = np.array(X)[np.array(y) == 0][1]
postive_inputs = feature_extractor(positive_entry.astype(float), sampling_rate=sampling_rate, return_tensors="pt")
negative_inputs = feature_extractor(negative_entry.astype(float), sampling_rate=sampling_rate, return_tensors="pt")
# Assuming inputs is your dictionary
positive_tensor_data = postive_inputs['input_values'].squeeze(0)  # Remove batch dimension if necessary (1, 40, 40 -> 40, 40)
negative_tensor_data = negative_inputs['input_values'].squeeze(0)  # Remove batch dimension if necessary (1, 40, 40 -> 40, 40)


inputs = feature_extractor(positive_entry.astype(float), sampling_rate=sampling_rate, return_tensors="np")

spectrogram_features = inputs['input_values'][0]  # Extracted feature matrix
plt.figure(figsize=(10,6))
plt.imshow(spectrogram_features.T, aspect='auto', origin='lower')
plt.colorbar(label='Intensity')
plt.ylabel('Mel Frequency Bin')
plt.xlabel('Time Frame')
plt.title('AST Feature Extractor Spectrogram')
plt.show()

# Create subplots
# fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# # Plot positive entry
# ax[0].imshow(positive_tensor_data, alpha=1.0)
# ax[0].set_title('Positive Entry')
# ax[0].axis('off')  # Optionally turn off axes

# # Plot negative entry
# ax[1].imshow(negative_tensor_data, alpha=1.0)
# ax[1].set_title('Negative Entry')
# ax[1].axis('off')  # Optionally turn off axes

# # Show the plot
# plt.colorbar(ax[0].images[0], ax=ax[0])
# plt.colorbar(ax[1].images[0], ax=ax[1])

# plt.tight_layout()
# plt.show()

(6000,)


C:\Users\NoahB\AppData\Local\Temp\ipykernel_18828\2668401813.py:36: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [3]:
# compute mean / std

import numpy as np
from tqdm import tqdm

extractor = ASTFeatureExtractor(
    sampling_rate=2000,         # Match your audio sampling rate
    num_mel_bins=128,           # Number of mel bins (adjustable)
    max_length=58,              # Based on your previous calculation
    do_normalize=False          # We compute mean and std, so disable for now
)

def compute_mean_std(extractor, dataset):
    all_features = []

    for audio in tqdm(dataset):
        # Ensure audio is in numpy array format
        if not isinstance(audio, np.ndarray):
            audio = np.array(audio)
        
        # Extract features using the extractor
        features = extractor._extract_fbank_features(audio, max_length=extractor.max_length)
        all_features.append(features)

    # Convert to a single array
    all_features = np.concatenate(all_features, axis=0)  # shape: (total_frames, num_mel_bins)

    # Compute mean and std
    mean = np.mean(all_features)
    std = np.std(all_features)

    print(f"Computed Mean: {mean}, Computed Std: {std}")
    return mean, std

mean, std = compute_mean_std(extractor, np.array(X).astype(float))
print(mean)
print(std)

100%|██████████████████████████████████████████████████████████████████████████████████████| 290/290 [00:00<00:00, 742.39it/s]


Computed Mean: -3.509511783990341, Computed Std: 12.877285997576628
-3.509511783990341
12.877285997576628


In [4]:
feature_extractor = ASTFeatureExtractor(
    sampling_rate=2000,
    num_mel_bins=128,  # Adjust as needed
    max_length=58,
    do_normalize=True,
    mean=mean,
    std=std
)

inputs = feature_extractor(positive_entry.astype(float), sampling_rate=sampling_rate, return_tensors="np")

spectrogram_features = inputs['input_values'][0]  # Extracted feature matrix
plt.figure(figsize=(10,6))
plt.imshow(spectrogram_features.T, aspect='auto', origin='lower')
plt.colorbar(label='Intensity')
plt.ylabel('Mel Frequency Bin')
plt.xlabel('Time Frame')
plt.title('AST Feature Extractor Spectrogram')
plt.show()

C:\Users\NoahB\AppData\Local\Temp\ipykernel_18828\2108109147.py:19: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [5]:
processed_Data_file_1 = r"C:\Users\NoahB\OneDrive\Desktop\cetacean_detection\nefsc_sbnms_200903_nopp6_ch10\processed\intial_run\images\NOPP6_EST_20090328_000000_CH10.npz"

data = np.load(processed_Data_file_1)
positive_tensor_data = data["X"][data["y"] == 1][1]
negative_tensor_data = data["X"][data["y"] == 0][1]

mean = np.mean(data["X"])
std = np.std(data["X"])
print("mean: ", mean)
print("std: ", std)
    
# Create subplots
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# Plot positive entry
ax[0].imshow(positive_tensor_data, alpha=1.0)
ax[0].set_title('Positive Entry')
ax[0].axis('off')  # Optionally turn off axes

# Plot negative entry
ax[1].imshow(negative_tensor_data, alpha=1.0)
ax[1].set_title('Negative Entry')
ax[1].axis('off')  # Optionally turn off axes

# Show the plot
plt.colorbar(ax[0].images[0], ax=ax[0])
plt.colorbar(ax[1].images[0], ax=ax[1])

plt.tight_layout()
plt.show()

mean:  37.29552868534483
std:  24.15963634614625


C:\Users\NoahB\AppData\Local\Temp\ipykernel_18828\3595545552.py:30: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Attempt to reproduce the spectrogram generation with torch. 

In [6]:
model = ASTForAudioClassification.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593", num_mel_bins = 400, max_length=400, ignore_mismatched_sizes=True)
# Customize the classifier head (change 10 to your desired number of classes)
num_classes = 2
model.classifier = torch.nn.Sequential(
    torch.nn.LayerNorm(768),
    torch.nn.Linear(768, num_classes)
)
model

positive_tensor_data = data["X"][data["y"] == 1][1]
positive_tensor_data = torch.from_numpy(positive_tensor_data.astype(np.float32)).unsqueeze(0)
positive_tensor_data.shape
inputs = {'input_values': positive_tensor_data}
with torch.no_grad():
        logits = model(**inputs).logits
print(logits)
# predicted_class_ids = torch.argmax(logits, dim=-1).item()
# predicted_label = model.config.id2label[predicted_class_ids]
# predicted_label

# # compute loss - target_label is e.g. "down"
# target_label = model.config.id2label[0]
# inputs["labels"] = torch.tensor([model.config.label2id[target_label]])
# loss = model(**inputs).loss
# print(round(loss.item(), 2))


Some weights of ASTForAudioClassification were not initialized from the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 and are newly initialized because the shapes did not match:
- audio_spectrogram_transformer.embeddings.position_embeddings: found shape torch.Size([1, 1214, 768]) in the checkpoint and torch.Size([1, 1523, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[-0.0733, -0.2175]])


Using the original ast model (pretrained with imagenet)

In [13]:
import sys
sys.path.append(r"C:\Users\NoahB\OneDrive\Desktop\cetacean_detection\ast\src\models")

from ast_models import ASTModel
ast_mdl = ASTModel(label_dim=2, input_tdim=400, input_fdim=400, model_size='tiny224')
# input a batch of 10 spectrogram, each with 100 time frames and 128 frequency bins
test_output = ast_mdl(positive_tensor_data)
# output should be in shape [10, 527], i.e., 10 samples, each with prediction of 527 classes.
print(test_output.shape)

---------------AST Model Summary---------------
ImageNet pretraining: True, AudioSet pretraining: False
frequncey stride=10, time stride=10
number of patches=1521
torch.Size([1, 2])
